In [12]:
from espn_api.basketball import League

league = League(
    league_id=136397, year=2022,
    espn_s2='AEA%2BfssjYA%2FMZuR3dTuXMRY%2BZIqS4LVJjMe%2FtUMJmm%2Bmwi9eXcJLFBJHz%2FLBfHuZmgwgGEy6JR1FTKtVA6wLt5Eqmwjp3I%2B2MlJFKr76Ooo0SsQjVU4D2gHwsjznWxrO54HQq1iBjqDBjbIt1TSUzk7AJygZGN2ENIXC%2F%2F4%2Boev1QZG8bcWhGdVKw%2BUU2ZYPALWiGNoTUDg9Yg%2B8w8mTkkKiUm4ilgzEo9EOBEZQr0OwRcfv400eLu59Mreu7mV8pXoUelvRKlUKnrOAy2wB%2BnOy',
    swid='{150E1CCD-71BB-410E-8E1C-CD71BB410ECA}'
)

# Per-team stats

In [117]:
from collections import defaultdict
import warnings
all_teams = []
keep_keys = ['PTS', 'BLK', "STL", "REB", 'AST', 'TO', 'FGM', 'FGA', 'FTM', 'FTA', '3PTM']

for team in league.teams:
#team = league.teams[0]
    projected_team_stats = defaultdict(int)
    projected_team_stats['team_id'] = team.team_id
    projected_team_stats['name'] = team.team_name
    for player in team.roster:
        # Last season stats are 002021
        # Projected season stats are 102022
        if '002021' in player.stats:
            stats_to_add = player.stats['002021']['avg']
        elif '102022' in player.stats:
            stats_to_add = player.stats['102022']['avg']
        else:
            warnings.warn(f"Can't find stats for player {player}")
        for k in keep_keys:
            projected_team_stats[k] += stats_to_add[k]
    projected_team_stats['FG%'] = projected_team_stats['FGM'] / projected_team_stats['FGA']
    projected_team_stats['FT%'] = projected_team_stats['FTM'] / projected_team_stats['FTA']
    all_teams.append(projected_team_stats)

/tmp/ipykernel_5136/3544169959.py:18: UserWarning: Can't find stats for player Player(Josh Giddey)
  warnings.warn(f"Can't find stats for player {player}")


In [119]:
import pandas as pd
team_stats = pd.DataFrame(all_teams)
team_stats

,team_id,name,PTS,BLK,STL,REB,AST,TO,FGM,FGA,FTM,FTA,3PTM,FG%,FT%
0,1,OG-Wan Kenobi,206.987446,8.741119,12.232657,85.437896,35.114640,20.652653,74.977997,154.353454,36.530143,43.705730,20.501310,0.485755,0.835820
1,2,Web Embiid,210.828727,8.222698,13.625778,73.264787,48.498503,27.689802,75.383852,160.056498,37.479788,45.386284,22.581235,0.470983,0.825795
2,3,Alex's NoChipsHere,208.727866,10.467462,13.612083,70.815759,42.061217,23.616770,76.536332,163.278579,29.774430,37.846511,25.880772,0.468747,0.786715
3,6,Team wu,232.266016,6.379566,12.675451,66.394683,54.091273,25.278233,82.813857,177.999575,39.631998,48.243334,27.006305,0.465247,0.821502
4,8,Big Brain Brand,206.993408,13.633182,14.229933,90.108333,41.123199,23.313154,76.153984,155.145183,35.364207,49.289445,19.321232,0.490856,0.717480
5,10,Agents of Hield,180.431698,8.591487,14.019397,83.911073,58.136892,23.103187,68.071325,131.997830,29.017468,35.272245,15.271579,0.515700,0.822671
6,16,Curry These nuts,213.059716,9.982534,15.072468,76.046727,51.966464,27.714488,77.362663,171.704674,31.840143,40.326110,26.494246,0.450557,0.789566
7,19,Team Lam,225.226960,7.770905,11.169588,82.495317,45.473866,24.387297,84.930827,174.956702,31.460963,41.867159,23.904343,0.485439,0.751447
8,20,Jonas Brothers,199.636213,8.205925,10.819611,73.981093,41.408525,25.509067,74.108976,158.606649,33.174150,40.850916,18.244110,0.467250,0.812078
9,22,Cookie Do,233.592948,9.279528,12.329069,84.809735,54.923391,27.999383,85.651435,176.980110,40.838655,52.420133,21.451424,0.483961,0.779064


# Professional schedules

In [133]:
league.firstScoringPeriod, league.scoringPeriodId, league.currentMatchupPeriod

(1, 1, 1)

In [168]:
scoring_id = league.current_week
league._get_pro_schedule(175)

{}

In [196]:
import datetime
all_schedules = {}

for scoring_id in range(1, 20):
    days_matchups = league._get_pro_schedule(scoring_id)
    for team_a, (team_b, timestamp) in days_matchups.items():
        date_object = (
            datetime.datetime.fromtimestamp(timestamp/1000)
            #.strftime("%Y-%m-%d")
        )
        if team_a not in all_schedules:
            all_schedules[team_a] = {}
        all_schedules[team_a][date_object] = 1

In [197]:
pro_schedules = pd.DataFrame(all_schedules)

In [203]:
date_filter = (
    (datetime.datetime(2021,10,18) < pro_schedules.index) & 
    (pro_schedules.index < datetime.datetime(2021, 10, 25))
    
)
pro_schedules[
    date_filter
]

,13,15,17,9,11,16,18,19,20,21,...,3,4,5,7,8,10,12,14,1,6
2021-10-19 22:00:00,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-22 22:00:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-24 21:30:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-19 19:30:00,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-21 20:00:00,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2021-10-23 20:30:00,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-22 19:30:00,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-24 16:00:00,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-21 22:00:00,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2021-10-24 21:00:00,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
team_id_name_mapping = {}
for team in data['settings']['proTeams']:
    team_id_name_mapping[team['id']] = team['abbrev'].upper()
    team_id_name_mapping[team['abbrev'].upper()] = team['id']
team_id_name_mapping[18] = 'NYK'
team_id_name_mapping['NYK'] = 18

team_id_name_mapping[20] = 'PHL'
team_id_name_mapping['PHL'] = 20

team_id_name_mapping[9] = 'GSW'
team_id_name_mapping['GSW'] = 9

In [234]:
specific_team = 1
matchup_stats = defaultdict(int)
games_played = defaultdict(int)
for player in league.teams[specific_team].roster:
    num_games = (
        pro_schedules
        .loc[
            date_filter, 
            team_id_name_mapping[player.proTeam.upper()]
        ].sum(skipna=True)
    )
    games_played[player.name] = num_games
    if '002021' in player.stats:
        stats_to_add = player.stats['002021']['avg']
    elif '102022' in player.stats:
        stats_to_add = player.stats['102022']['avg']
    else:
        warnings.warn(f"Can't find stats for player {player}")
    for k in keep_keys:
        matchup_stats[k] += num_games * stats_to_add[k]
matchup_stats['FG%'] = matchup_stats['FGM'] / matchup_stats['FGA']
matchup_stats['FT%'] = matchup_stats['FTM'] / matchup_stats['FTA']

In [235]:
matchup_stats

defaultdict(int,
            {'PTS': 590.3310680245205,
             'BLK': 22.704404180966495,
             'STL': 37.699995738148075,
             'REB': 198.4661093908106,
             'AST': 131.86930401204393,
             'TO': 76.55730110636735,
             'FGM': 211.0634732884769,
             'FGA': 448.2558280476959,
             'FTM': 105.4325666050355,
             'FTA': 127.126536426463,
             '3PTM': 62.77155484253124,
             'FG%': 0.4708549450605672,
             'FT%': 0.8293513657238945})

In [236]:
games_played

defaultdict(int,
            {'Jayson Tatum': 3.0,
             'Joel Embiid': 3.0,
             'Jaylen Brown': 3.0,
             'Khris Middleton': 3.0,
             'Kyle Lowry': 2.0,
             'Jusuf Nurkic': 2.0,
             'Darius Garland': 3.0,
             'Marcus Smart': 3.0,
             'Kelly Olynyk': 2.0,
             'Nerlens Noel': 3.0,
             'Seth Curry': 3.0,
             'Isaiah Roby': 3.0,
             'Jordan Poole': 3.0})